In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'2.0.21'

In [16]:
from sqlalchemy.orm import Session

from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

In [17]:
from sqlalchemy import create_engine
from sqlalchemy import text

from sqlalchemy import String
from sqlalchemy import ForeignKey

In [18]:
from typing import List
from typing import Optional



# engine (always needed?)

In [7]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

# type of DB  +  BDAPI :///: DB location

engine

Engine(sqlite+pysqlite:///:memory:)

# using Session, execute, commit and result

In [ ]:
with Session(engine) as session:
    session.execute(text("CREATE TABLE some_table (x int, y int)"))

In [13]:
# need to commit as we go, otherwise rollback
with Session(engine) as session:
    session.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    session.commit()

2023-10-31 13:26:04,606 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:26:04,607 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-10-31 13:26:04,608 INFO sqlalchemy.engine.Engine [cached since 68.33s ago] [(1, 1), (2, 4)]
2023-10-31 13:26:04,610 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 1})
    for row in result:
        print(f"\tx: {row.x}  y: {row.y}")

2023-10-31 13:26:13,452 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:26:13,453 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-10-31 13:26:13,454 INFO sqlalchemy.engine.Engine [cached since 133.4s ago] (1,)
	x: 2  y: 4
2023-10-31 13:26:13,457 INFO sqlalchemy.engine.Engine ROLLBACK


# MetaData

In [19]:
class Base(DeclarativeBase):
    pass

In [20]:
class User(Base):
    __tablename__ = "user_account"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]   # can be null
        
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id!r}), name={self.name!r}, fullname={self.fullname!r})"
    
class Address(Base):
    __tablename__ = "address"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    email_address: Mapped[str]
        
    user: Mapped[User] = relationship(back_populates="addresses")
        
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [22]:
for t in Base.metadata.sorted_tables:
    print(t)

user_account
address


In [23]:
Base.metadata.create_all(engine)

2023-10-31 13:29:58,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-31 13:29:58,716 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-10-31 13:29:58,717 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 13:29:58,719 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-10-31 13:29:58,720 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 13:29:58,721 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-10-31 13:29:58,721 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 13:29:58,722 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-10-31 13:29:58,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-31 13:29:58,726 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-10-31 13:29:58,726 INFO sqlalchemy.engine.Engine [no key 0.00087s] ()
2023-10-31 13:29:58,728 INFO sqlalchemy.engine.